In [2]:
import text_lloom.src.text_lloom.workbench as wb
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
from text_lloom.src.text_lloom.llm import OpenAIModel


In [3]:
def load_and_combine_data(basedir='/home/takeh/public_health/reports/cluster_comments/'):
    """すべてのクラスターのCSVファイルを読み込み、統合したDataFrameを作成"""
    
    cluster_files = [
        'cluster0_comments.csv',
        'cluster1_comments.csv', 
        'cluster2_comments.csv',
        'cluster3_comments.csv',
        'cluster4_comments.csv'
    ]
    
    all_data = []
    
    for i, file_name in enumerate(cluster_files):
        try:
            # フルパスを構築
            full_path = os.path.join(basedir, file_name)
            print(f"読み込み中: {full_path}")
            df = pd.read_csv(full_path)
            
            # 空でないかチェック
            if len(df) == 0:
                print(f"{file_name} は空のファイルです。スキップします。")
                continue
                
            # クラスター番号を追加
            df['cluster'] = f'cluster_{i}'
            df['cluster_id'] = i
            
            # ユニークなIDを作成（累積インデックス）
            df['doc_id'] = df.index + sum(len(data) for data in all_data)
            
            all_data.append(df)
            print(f"✓ {file_name}: {len(df)}件のコメントを読み込み")
            
        except FileNotFoundError:
            print(f"✗ {full_path} が見つかりません。")
        except Exception as e:
            print(f"✗ {file_name} の読み込みエラー: {e}")
    
    # 全データを統合
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\n✓ 統合完了: 合計 {len(combined_df)} 件のコメント")
        print(f"クラスター分布:\n{combined_df['cluster'].value_counts()}")
        return combined_df
    else:
        print("✗ 読み込み可能なデータがありませんでした。")
        return None

In [4]:
df = load_and_combine_data()

読み込み中: /home/takeh/public_health/reports/cluster_comments/cluster0_comments.csv
✓ cluster0_comments.csv: 398件のコメントを読み込み
読み込み中: /home/takeh/public_health/reports/cluster_comments/cluster1_comments.csv
✓ cluster1_comments.csv: 4452件のコメントを読み込み
読み込み中: /home/takeh/public_health/reports/cluster_comments/cluster2_comments.csv
✓ cluster2_comments.csv: 90件のコメントを読み込み
読み込み中: /home/takeh/public_health/reports/cluster_comments/cluster3_comments.csv
✓ cluster3_comments.csv: 186件のコメントを読み込み
読み込み中: /home/takeh/public_health/reports/cluster_comments/cluster4_comments.csv
✓ cluster4_comments.csv: 574件のコメントを読み込み

✓ 統合完了: 合計 5700 件のコメント
クラスター分布:
cluster
cluster_1    4452
cluster_4     574
cluster_0     398
cluster_3     186
cluster_2      90
Name: count, dtype: int64


In [5]:
df

,UserID,comments,cluster,cluster_id,doc_id
0,UCaLJHKwN9DbuWmNQlS5vkog,こんばんは女性若いアラサー患者は子宮かん乳ガン受け人無理,cluster_0,0,0
1,UCaLJHKwN9DbuWmNQlS5vkog,こんばんは若い女性は二十歳三十歳四十歳アラサー多くは,cluster_0,0,1
2,UCaLJHKwN9DbuWmNQlS5vkog,こんばんは女性患者子宮頚若い二十歳三十歳来れ将来結婚子供,cluster_0,0,2
3,UCaLJHKwN9DbuWmNQlS5vkog,私は間近周辺人女性病気婦人淋しいいません職場転職ますおもい子宮けいがんは,cluster_0,0,3
4,UCaLJHKwN9DbuWmNQlS5vkog,こんばんは子宮痙かん女性患者は正しい情報,cluster_0,0,4
...,...,...,...,...,...
5695,UCP2viLKgu1VQezTYEPOpysg,私は3年前に子宮頚がんの検査を初めて受けて、引っかかり精密検査受けると軽度異形成と診断されま...,cluster_4,4,5695
5696,UC6jE6ebLoaYPULWKI6_rzNg,軽度異形成になったことがあります。\n組織診は組織を採るから痛かったけど\n普通の検査はブラ...,cluster_4,4,5696
5697,UCsCbZ2LsnaK_55UE3G5uKJA,大袈裟やな,cluster_4,4,5697
5698,UCEiBNKydrDSS1EaQAYKjwLg,私も数年前に検査に引っかかり泣きながら精密検査に行ったことがあります。幸い異形成の初期で1年...,cluster_4,4,5698


In [6]:
gpt41mini = OpenAIModel(name="gpt-4.1-mini",
api_key=os.environ["OPENAI_API_KEY"])
gpt41nano = OpenAIModel(name="gpt-4.1-nano",
api_key=os.environ["OPENAI_API_KEY"])

l = wb.lloom(
    df=df,
    text_col="comments",
    id_col="doc_id",
    distill_model=gpt41mini, 
    score_model=gpt41nano     
)

In [7]:
comment_params = {
    "filter_n_quotes": 2,    
    "summ_n_bullets": 2,     
    "synth_n_concepts": 5    
}

In [8]:
await l.gen(params=comment_params)



Estimated cost: $2.24
**Please note that this is only an approximate cost estimate**


Action required


Proceed with generation? (y/n):  y




Distill-filter
⠼ LoadingERROR json_load on: {
    "relevant_quotes": [ "接種しとくかぁ", "（）」 ]
}
✅ Done    


,doc_id,comments
0,0,女性若いアラサー患者\n子宮かん乳ガン
1,1,こんばんは\n若い女性は二十歳三十歳四十歳アラサー多くは
2,2,女性患者\n子宮頚
3,3,子宮けいがんは\n職場転職ます
4,4,子宮痙かん\n女性患者は正しい情報
...,...,...
5694,5695,私は3年前に子宮頚がんの検査を初めて受けて、引っかかり精密検査受けると軽度異形成と診断されま...
5695,5696,軽度異形成になったことがあります。\n普通の検査はブラシで擦るくらいで痛くないと個人的に感じ...
5696,5697,大袈裟\nやな
5697,5698,私も数年前に検査に引っかかり泣きながら精密検査に行ったことがあります。\n幸い異形成の初期で...




Distill-summarize
✅ Done    


...

⠦ Loading 

/home/takeh/lloom/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/takeh/lloom/.venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ Done    


,doc_id,comments,cluster_id
11368,5683,Planning to get first dose tomorrow,-1
11367,5683,Unaware of new 9-valent vaccine,-1
10053,5026,Had a cone biopsy procedure,-1
10079,5039,Expressing gratitude to Kirichan,-1
10089,5044,Could not get vaccinated at that time,-1
...,...,...,...
11397,5698,数年前に検査で異常が見つかる,1
11398,5698,初期異形成は1年で消失,1
11399,5699,母が乳がんで抗がん剤治療経験,1
11400,5699,自分も検診に行く決意,1




Synthesize
⠼ LoadingERROR json_load on: The text examples provided are a collection of summaries, each associated with a unique `example_id`. These summaries cover a wide range of topics, including medical conditions, treatments, and societal issues. Here are some patterns and themes that can be identified from the examples:

1. **Medical Conditions and Treatments**: Many examples discuss various medical conditions, particularly related to cancer (e.g., cervical cancer, breast cancer), and the importance of early detection and regular check-ups. There are mentions of specific diagnostic tests, treatments, and the experiences of patients undergoing these procedures.

2. **Vaccination and Public Health**: A significant number of examples focus on HPV vaccination, its importance, and the public's perception of it. There are discussions about the benefits and risks associated with vaccines, as well as the societal and governmental roles in promoting vaccination.

3. **Gender and Health**

In [9]:
l.summary()

Total time: 475.15 sec (7.92 min)
	('Distill-filter', '2025-07-31-13-50-06'): 185.94 sec
	('Distill-summarize', '2025-07-31-13-53-11'): 184.97 sec
	('Cluster', '2025-07-31-13-54-22'): 70.61 sec
	('Synthesize', '2025-07-31-13-54-50'): 28.01 sec
	('Review-remove', '2025-07-31-13-54-52'): 2.61 sec
	('Review-merge', '2025-07-31-13-54-55'): 3.02 sec


Total cost: $2.70
	('Distill-filter', '2025-07-31-13-50-06'): $0.870
	('Distill-summarize', '2025-07-31-13-53-11'): $0.631
	('Synthesize', '2025-07-31-13-54-50'): $1.188
	('Review-remove', '2025-07-31-13-54-52'): $0.002
	('Review-merge', '2025-07-31-13-54-55'): $0.004


Tokens: total=2462410, in=1952201, out=510209


In [10]:
l.select()

ConceptSelectWidget(data='{"5711b6ed-83b3-45dc-a1c5-4567cd6c903b": {"id": "5711b6ed-83b3-45dc-a1c5-4567cd6c903…

In [11]:
l.save(folder="./lloom_comments", file_name="before_scoring")

Saved session to ./lloom_comments/before_scoring.pkl


In [13]:
import pandas as pd
import json
import pickle
from datetime import datetime

# タイムスタンプ付きフォルダ
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = f"./lloom_backup_{timestamp}"
import os
os.makedirs(save_dir, exist_ok=True)

# 1. 生成されたコンセプトをJSON保存
concepts_data = {}
for c_id, concept in l.concepts.items():
  concepts_data[c_id] = concept.to_dict()

with open(f"{save_dir}/concepts.json", "w", encoding="utf-8") as f:
  json.dump(concepts_data, f, ensure_ascii=False, indent=2)

# 2. フィルター結果をCSV保存
if hasattr(l, 'df_filtered') and l.df_filtered is not None:
  l.df_filtered.to_csv(f"{save_dir}/filtered_data.csv",
index=False, encoding="utf-8")
  print(f"フィルター結果保存: {len(l.df_filtered)} rows")

# 3. 要約結果をCSV保存
if hasattr(l, 'df_bullets') and l.df_bullets is not None:
  l.df_bullets.to_csv(f"{save_dir}/bullet_summaries.csv",
index=False, encoding="utf-8")
  print(f"要約結果保存: {len(l.df_bullets)} rows")

# 4. 元データもバックアップ
l.in_df.to_csv(f"{save_dir}/original_data.csv", index=False,
encoding="utf-8")

フィルター結果保存: 5699 rows
要約結果保存: 11401 rows


In [14]:
  backup_info = {
      "timestamp": timestamp,
      "original_data_shape": l.in_df.shape,
      "filtered_data_shape": l.df_filtered.shape if hasattr(l,
  'df_filtered') and l.df_filtered is not None else None,
      "bullet_data_shape": l.df_bullets.shape if hasattr(l,
  'df_bullets') and l.df_bullets is not None else None,
      "num_concepts": len(l.concepts),
      "concept_names": [c.name for c in l.concepts.values()],
      "text_column": l.doc_col,
      "id_column": l.doc_id_col,
      "models": {
          "distill_model": l.distill_model.name if
  hasattr(l.distill_model, 'name') else str(l.distill_model),
          "score_model": l.score_model.name if
  hasattr(l.score_model, 'name') else str(l.score_model),
          "synth_model": l.synth_model.name if
  hasattr(l.synth_model, 'name') else str(l.synth_model),
          "cluster_model": l.cluster_model.name if
  hasattr(l.cluster_model, 'name') else str(l.cluster_model),
      }
  }

  with open(f"{save_dir}/backup_info.json", "w", encoding="utf-8") as f:
      json.dump(backup_info, f, ensure_ascii=False, indent=2)

  print("設定情報も保存完了")

設定情報も保存完了


In [ ]:
score_df = await l.score(
    score_all=True,
    sequential_processing=True,  # 逐次処理
    pivot_output=True,
    get_highlights=True,
    batch_size=3,
    debug=True  # 進捗表示を有効化
)



Scoring 6 concepts for 5699 documents (sequentially (one by one))
Estimated cost: $1.44
Note: Sequential processing provides better progress visibility and error handling
Note: Output will be in pivot format (one row per document with all concept scores)
**Please note that this is only an approximate cost estimate**


Action required


Proceed with scoring? (y/n):  y


In [ ]:
import builtins
import sys

# inputを一時的にオーバーライド
def auto_confirm(prompt=""):
    print(f"{prompt}y", flush=True)
    return "y"

original_input = builtins.input
builtins.input = auto_confirm

try:
    score_df = await l.score(
        score_all=True,
        sequential_processing=True,
        pivot_output=True,
        get_highlights=True,
        batch_size=5,
        debug=True  # 進捗表示は維持
    )
finally:
    builtins.input = original_input

In [2]:
import text_lloom.src.text_lloom.workbench as wb
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
from text_lloom.src.text_lloom.llm import OpenAIModel


In [3]:
# 1. 修正が適用されているか確認
import importlib
import text_lloom.src.text_lloom.concept_induction as ci
importlib.reload(ci)  # モジュールをリロード

<module 'text_lloom.src.text_lloom.concept_induction' from '/home/takeh/lloom/text_lloom/src/text_lloom/concept_induction.py'>

In [4]:
# 後で復元する場合
import pickle

# セッション全体の復元
with open("./lloom_comments/before_scoring.pkl", "rb") as f:
  l = pickle.load(f)


l.select()

ConceptSelectWidget(data='{"5711b6ed-83b3-45dc-a1c5-4567cd6c903b": {"id": "5711b6ed-83b3-45dc-a1c5-4567cd6c903…

In [5]:
new_score_model = OpenAIModel(
  name="gpt-4.1-nano",  # または他のモデル
  api_key=os.environ["OPENAI_API_KEY"]
)

# 既存のインスタンスのスコアリングモデルを変更
l.score_model = new_score_model



In [ ]:
await l.add(
    # Your new concept name
    name="ワクチン接種批判",
    # Your new concept prompt
    prompt="この文章はワクチン接種に対しての批判をしているか", 
)



Scoring 1 concepts for 5699 documents (in parallel)
Estimated cost: $0.39
**Please note that this is only an approximate cost estimate**


Action required


Proceed with scoring? (y/n):  y


  0%|                                                                                                                                         | 0/1 [00:00<?, ?it/s]

In [ ]:
l.select()

In [7]:
score_df = await l.score(
  score_all=True,
  sequential_processing=True, 
  pivot_output=True,
  get_highlights=True,
  batch_size=3,
  debug=True  
)



Scoring 6 concepts for 5699 documents (sequentially (one by one))
Estimated cost: $1.44
Note: Sequential processing provides better progress visibility and error handling
Note: Output will be in pivot format (one row per document with all concept scores)
**Please note that this is only an approximate cost estimate**


Action required


Proceed with scoring? (y/n):  y



Sequential concept scoring: Processing 6 concepts one by one

Processing concept 1/6: Medical Diagnosis
ERROR json_load on: {
    "pattern_results": [
        {
            "example_id": "18",
            "rationale": "The example mentions \u5b50\u5bae\u9838\u304c\u3093\u30ef\u30af\u30c1\u30f3\u and \u5c40\u90e8\u969c\u5bb3, which relate to health issues but do not specify a diagnosis or procedures.",
            "answer": "D",
            "quote": "\u5b50\u5bae\u9838\u304c\u3093\u30ef\u30af\u30c1\u30f3\n\u5c40\u90e8\u969c\u5bb3"
        },
        {
            "example_id": "19",
            "rationale": "The text mentions \u304c\u3093\u30ce\u30fc\u30c8\u60a3\u8005\u4f1a and \u5fdc\u63f4\u307e\u3059, indicating support for a patient group but not referencing any diagnosis or procedures.",
            "answer": "D",
            "quote": "\u304c\u3093\u30ce\u30fc\u30c8\u60a3\u8005\u4f1a\n\u5fdc\u63f4\u307e\u3059"
        },
        {
            "example_id": "20",
            "ration

In [8]:
if 'score_df' in locals():
  # スコア結果が既にある場合
  score_df.to_csv("lloom_scores_latest.csv", index=False,
encoding="utf-8-sig")
  print("保存完了: lloom_scores_latest.csv")
else:
  print("まずスコアリングを完了してください")

保存完了: lloom_scores_latest.csv


In [9]:
score_df

,doc_id,text,concept1_id,concept1_name,concept1_prompt,concept1_score,concept1_rationale,concept1_highlight,concept1_seed,concept2_id,...,concept5_rationale,concept5_highlight,concept5_seed,concept6_id,concept6_name,concept6_prompt,concept6_score,concept6_rationale,concept6_highlight,concept6_seed
0,0,\u5973\u6027\u82e5\u3044\u30a2\u30e9\u30b5\u30...,5711b6ed-83b3-45dc-a1c5-4567cd6c903b,Medical Diagnosis,"Does the text mention a medical diagnosis, suc...",0.25,The text mentions a disease related to women a...,\u5973\u6027\u82e5\u3044\u30a2\u30e9\u30b5\u30...,NaN,644481e1-85f4-4ccd-91ee-e0170e4872fd,...,The example discusses a female patient and her...,\u5973\u6027\u82e5\u3044\u30a2\u30e9\u30b5\u30...,NaN,92ab2c05-358b-4fb7-8a69-65b106a057ab,Health and Support,Does the text emphasize the importance of heal...,1.00,The example mentions a female patient and a pe...,\u5973\u6027\u82e5\u3044\u30a2\u30e9\u30b5\u30...,NaN
1,1,\u3053\u3093\u3070\u3093\u306f\n\u82e5\u3044\u...,5711b6ed-83b3-45dc-a1c5-4567cd6c903b,Medical Diagnosis,"Does the text mention a medical diagnosis, suc...",0.25,The text describes age and gender but does not...,\u3053\u3093\u3070\u3093\u306f\n\u82e5\u3044\u...,NaN,644481e1-85f4-4ccd-91ee-e0170e4872fd,...,The example mentions a young woman and her age...,\u82e5\u3044\u5973\u6027\u306f\u4e8c\u5341\u6b...,NaN,92ab2c05-358b-4fb7-8a69-65b106a057ab,Health and Support,Does the text emphasize the importance of heal...,1.00,The text discusses a young woman and a pediatr...,\u82e5\u3044\u5973\u6027\u306f\u4e8c\u5341\u6b...,NaN
2,2,\u5973\u6027\u60a3\u8005\n\u5b50\u5bae\u981a,5711b6ed-83b3-45dc-a1c5-4567cd6c903b,Medical Diagnosis,"Does the text mention a medical diagnosis, suc...",0.25,The text mentions a disease and a location but...,\u5973\u6027\u60a3\u8005\n\u5b50\u5bae\u981a,NaN,644481e1-85f4-4ccd-91ee-e0170e4872fd,...,The example states 'female patient' and 'child...,\u5973\u6027\u60a3\u8005\n\u5b50\u5bae\u981a,NaN,92ab2c05-358b-4fb7-8a69-65b106a057ab,Health and Support,Does the text emphasize the importance of heal...,1.00,The phrase 'female patient' and 'pediatrician'...,\u5973\u6027\u60a3\u8005\n\u5b50\u5bae\u981a,NaN
3,3,\u5b50\u5bae\u3051\u3044\u304c\u3093\u306f\n\u...,5711b6ed-83b3-45dc-a1c5-4567cd6c903b,Medical Diagnosis,"Does the text mention a medical diagnosis, suc...",0.25,The example mentions a medical facility and a ...,\u5b50\u5bae\u3051\u3044\u304c\u3093\u306f\n\u...,NaN,644481e1-85f4-4ccd-91ee-e0170e4872fd,...,The example discusses a career change and work...,\u5b50\u5bae\u3051\u3044\u304c\u3093\u306f\n\u...,NaN,92ab2c05-358b-4fb7-8a69-65b106a057ab,Health and Support,Does the text emphasize the importance of heal...,1.00,The example mentions a health checkup and a pr...,\u5b50\u5bae\u3051\u3044\u304c\u3093\u306f\n\u...,NaN
4,4,\u5b50\u5bae\u75d9\u304b\u3093\n\u5973\u6027\u...,5711b6ed-83b3-45dc-a1c5-4567cd6c903b,Medical Diagnosis,"Does the text mention a medical diagnosis, suc...",0.25,The example references a woman patient and a c...,\u5b50\u5bae\u75d9\u304b\u3093\n\u5973\u6027\u...,NaN,644481e1-85f4-4ccd-91ee-e0170e4872fd,...,The example mentions a woman patient and infor...,\u5b50\u5bae\u75d9\u304b\u3093\n\u5973\u6027\u...,NaN,92ab2c05-358b-4fb7-8a69-65b106a057ab,Health and Support,Does the text emphasize the importance of heal...,1.00,The example discusses a health checkup and men...,\u5b50\u5bae\u75d9\u304b\u3093\n\u5973\u6027\u...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5694,4098,\u598a\u6d3b\u59cb\u3081\u305f\u3068\u3057\u30...,5711b6ed-83b3-45dc-a1c5-4567cd6c903b,Medical Diagnosis,"Does the text mention a medical diagnosis, suc...",0.00,,,NaN,644481e1-85f4-4ccd-91ee-e0170e4872fd,...,The text describes a personal activity related...,\u598a\u6d3b\u59cb\u3081\u305f\u3068\u3057\u30...,NaN,92ab2c05-358b-4fb7-8a69-65b106a057ab,Health and Support,Does the text emphasize the importance of heal...,0.00,,,NaN
5695,4271,\u79c1\u3082\u5b50\u5bae\u4f53\u7

In [21]:
l.summary()

Total time: 4145.82 sec (69.10 min)
	('Distill-filter', '2025-07-31-13-50-06'): 185.94 sec
	('Distill-summarize', '2025-07-31-13-53-11'): 184.97 sec
	('Cluster', '2025-07-31-13-54-22'): 70.61 sec
	('Synthesize', '2025-07-31-13-54-50'): 28.01 sec
	('Review-remove', '2025-07-31-13-54-52'): 2.61 sec
	('Review-merge', '2025-07-31-13-54-55'): 3.02 sec
	('Score', '2025-07-31-15-20-41'): 3670.66 sec


Total cost: $5.59
	('Distill-filter', '2025-07-31-13-50-06'): $0.870
	('Distill-summarize', '2025-07-31-13-53-11'): $0.631
	('Synthesize', '2025-07-31-13-54-50'): $1.188
	('Review-remove', '2025-07-31-13-54-52'): $0.002
	('Review-merge', '2025-07-31-13-54-55'): $0.004
	('Score-helper', '2025-07-31-14-25-42'): $0.466
	('Score-helper', '2025-07-31-14-38-38'): $0.525
	('Score-helper', '2025-07-31-14-45-56'): $0.459
	('Score-helper', '2025-07-31-15-01-14'): $0.507
	('Score-helper', '2025-07-31-15-12-52'): $0.472
	('Score-helper', '2025-07-31-15-20-40'): $0.463


Tokens: total=16278968, in=10731101, 

In [20]:
from text_lloom.src.text_lloom.llm import OpenAIModel, OpenAIEmbedModel
# API keyを確認
api_key = os.environ.get("OPENAI_API_KEY")
# モデルを再設定
l.distill_model = OpenAIModel(name="gpt-4.1-mini",api_key=api_key)
l.cluster_model =OpenAIEmbedModel(name="text-embedding-3-large", api_key=api_key)
l.synth_model = OpenAIModel(name="gpt-4.1-mini", api_key=api_key)
l.score_model = OpenAIModel(name="gpt-4.1-nano",api_key=api_key)

print("モデル再設定完了")

モデル再設定完了
